In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# General imports
import cortex
import importlib
import ipdb
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append("{}/../../../../utils".format(os.getcwd()))
from pycortex_utils import draw_cortex, set_pycortex_config_file,load_surface_pycortex


In [2]:
with open('../../../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)



In [ ]:
# main_dir = '/Users/uriel/disks/meso_shared'
main_dir = '/home/ulascombes/disks/meso_shared'
project_dir = 'RetinoMaps'
pycortex_subject = 'sub-170k'
format_ = '170k'
subject = 'sub-170k'
# Maps settings
rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
    amp_idx, baseline_idx, x_idx, y_idx = 0,1,2,3,4,5,6,7,8
      
cmap_polar, cmap_uni, cmap_ecc_size = 'hsv', 'Reds', 'Spectral'
col_offset = 1.0/14.0
cmap_steps = 255

description_end = 'avg gridfit'
deriv_fn_label = 'avg-gridfit'

# plot scales
rsq_scale = [0, 1]
ecc_scale = [0, 7.5]
size_scale = [0, 7.5]

# Set pycortex db and colormaps
cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
set_pycortex_config_file(cortex_dir)
importlib.reload(cortex)
 

# Define directories and fn


data_dir = '{}/{}/derivatives/pp_data/sub-170k/170k/prf_derivatives'.format(main_dir, project_dir)
data_fn = '{}_task-pRF_fmriprep_dct_avg_prf-deriv_gauss_gridfit.dtseries.nii'.format(subject)


results = load_surface_pycortex(brain_fn='{}/{}'.format(data_dir,data_fn))
deriv_mat = results['data_concat']
save_svg = False

print('Creating flatmaps...')

maps_names = []

# threshold data
deriv_mat_th = deriv_mat
amp_down =  deriv_mat_th[amp_idx,...] > 0
rsqr_th_down = deriv_mat_th[rsq_idx,...] >= analysis_info['rsqr_th'][0]
rsqr_th_up = deriv_mat_th[rsq_idx,...] <= analysis_info['rsqr_th'][1]
size_th_down = deriv_mat_th[size_idx,...] >= analysis_info['size_th'][0]
size_th_up = deriv_mat_th[size_idx,...] <= analysis_info['size_th'][1]
ecc_th_down = deriv_mat_th[ecc_idx,...] >= analysis_info['ecc_th'][0]
ecc_th_up = deriv_mat_th[ecc_idx,...] <= analysis_info['ecc_th'][1]
all_th = np.array((amp_down,rsqr_th_down,rsqr_th_up,size_th_down,size_th_up,ecc_th_down,ecc_th_up)) 
deriv_mat[rsq_idx,np.logical_and.reduce(all_th)==False]=0

# r-square
rsq_data = deriv_mat[rsq_idx,...]
alpha_range = analysis_info["alpha_range"]
alpha = (rsq_data - alpha_range[0])/(alpha_range[1]-alpha_range[0])
alpha[alpha>1]=1
param_rsq = {'data': rsq_data, 'cmap': cmap_uni, 'alpha': rsq_data, 
             'vmin': rsq_scale[0], 'vmax': rsq_scale[1], 'cbar': 'discrete', 
             'cortex_type': 'VertexRGB','description': 'pRF rsquare',
             'curv_brightness': 1, 'curv_contrast': 0.1, 'add_roi': save_svg,
             'cbar_label': 'pRF R2', 'with_labels': True}
maps_names.append('rsq')

# polar angle
pol_comp_num = deriv_mat[polar_real_idx,...] + 1j * deriv_mat[polar_imag_idx,...]
polar_ang = np.angle(pol_comp_num)
ang_norm = (polar_ang + np.pi) / (np.pi * 2.0)
ang_norm = np.fmod(ang_norm + col_offset,1)
param_polar = {'data': ang_norm, 'cmap': cmap_polar, 'alpha': alpha, 
               'vmin': 0, 'vmax': 1, 'cmap_steps': cmap_steps, 'cortex_type': 'VertexRGB',
               'cbar': 'polar', 'col_offset': col_offset, 
               'description': 'pRF polar:{:3.0f} steps{}'.format(cmap_steps, description_end), 
               'curv_brightness': 0.1, 'curv_contrast': 0.25, 'add_roi': save_svg, 
               'with_labels': True}
exec('param_polar_{cmap_steps} = param_polar'.format(cmap_steps = int(cmap_steps)))
exec('maps_names.append("polar_{cmap_steps}")'.format(cmap_steps = int(cmap_steps)))

# eccentricity
ecc_data = deriv_mat[ecc_idx,...]
param_ecc = {'data': ecc_data, 'cmap': cmap_ecc_size, 'alpha': alpha,
             'vmin': ecc_scale[0], 'vmax': ecc_scale[1], 'cbar': 'ecc', 'cortex_type': 'VertexRGB',
             'description': 'pRF eccentricity{}'.format(description_end), 'curv_brightness': 1,
             'curv_contrast': 0.1, 'add_roi': save_svg, 'with_labels': True}

maps_names.append('ecc')

# size
size_data = deriv_mat[size_idx,...]
param_size = {'data': size_data, 'cmap': cmap_ecc_size, 'alpha': alpha, 
              'vmin': size_scale[0], 'vmax': size_scale[1], 'cbar': 'discrete', 
              'cortex_type': 'VertexRGB', 'description': 'pRF size{}'.format(description_end), 
              'curv_brightness': 1, 'curv_contrast': 0.1, 'add_roi': False, 'cbar_label': 'pRF size',
              'with_labels': True}
maps_names.append('size')



# draw flatmaps
volumes = {}
for maps_name in maps_names:

    # create flatmap
    roi_name = 'pRF_{}'.format(maps_name)
    roi_param = {'subject': pycortex_subject, 'xfmname': None, 'roi_name': roi_name}
    print(roi_name)
    exec('param_{}.update(roi_param)'.format(maps_name))
    exec('volume_{maps_name} = draw_cortex(**param_{maps_name})'.format(maps_name = maps_name))

    

    
